<a href="https://colab.research.google.com/github/JamieNguru/Quantum-Solvers/blob/main/quantum_time_capsule_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install qiskit qiskit-ibm-runtime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.7/368.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.6 MB/s eta 0:00:00


In [ ]:
!pip install qiskit streamlit gTTS matplotlib pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0


In [ ]:
import os

# Define base and subfolder paths
BASE_DIR = '/content/quantum_time_capsule'
ASSETS_DIR = os.path.join(BASE_DIR, 'assets')

# Create the folders if they don't exist
os.makedirs(ASSETS_DIR, exist_ok=True)

# Confirm
print("Project folder created at:", BASE_DIR)
print("Assets folder created at:", ASSETS_DIR)


Project folder created at: /content/quantum_time_capsule
Assets folder created at: /content/quantum_time_capsule/assets


In [ ]:
%%writefile /content/quantum_time_capsule/quantum_scenes.py
print("Hello from quantum_scenes.py")


Writing /content/quantum_time_capsule/quantum_scenes.py


In [ ]:
%%writefile /content/quantum_time_capsule/app.py
print("Hello from Streamlit app")


Writing /content/quantum_time_capsule/app.py


In [ ]:
!pip install qiskit qiskit-aer --quiet


In [ ]:
%%writefile /content/quantum_time_capsule/quantum_scenes.py

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer, AerSimulator
import matplotlib
matplotlib.use("Agg")  # ensure non-GUI backend
import matplotlib.pyplot as plt
import os, random
from collections import OrderedDict

# Initialize backend
try:
    _BACKEND = Aer.get_backend('aer_simulator')
except Exception:
    _BACKEND = Aer.get_backend('qasm_simulator')

BASE_DIR = '/content/quantum_time_capsule'
ASSETS_DIR = os.path.join(BASE_DIR, 'assets')
circuit_path = os.path.join(ASSETS_DIR, 'double_slit_circuit.png')
os.makedirs(ASSETS_DIR, exist_ok=True)

# Base paths (allow override with env var)
BASE_DIR = os.environ.get('QT_CAPSULE_DIR', '/content/quantum_time_capsule')
ASSETS_DIR = os.path.join(BASE_DIR, 'assets')
os.makedirs(ASSETS_DIR, exist_ok=True)


def _save_histogram(counts, filepath):
    """
    Try plot_histogram; if that fails, draw a simple matplotlib bar plot.
    Returns the matplotlib.figure.Figure instance.
    """
    try:
        fig = plot_histogram(counts)
        fig.savefig(filepath, bbox_inches='tight')
        # don't close — caller may want to return fig to Streamlit
        return fig
    except Exception:
        labels = list(counts.keys())
        values = [counts[k] for k in labels]
        fig, ax = plt.subplots()
        ax.bar(labels, values)
        ax.set_xlabel('Outcome')
        ax.set_ylabel('Counts')
        plt.tight_layout()
        fig.savefig(filepath)
        return fig


def _save_circuit(qc, filepath):
    """
    Save a quantum circuit as an image (if matplotlib drawer works),
    otherwise fall back to ASCII text.
    """
    try:
        circ_fig = qc.draw(output='mpl', scale=1.2)
        circ_fig.savefig(filepath, bbox_inches='tight')
        print(f"✅ Saved circuit image to {os.path.abspath(filepath)}")
        return filepath
    except Exception as e:
        print("⚠️ Circuit image save failed:", e)
        txt_path = filepath.rsplit('.', 1)[0] + '.txt'
        try:
            ascii_drawing = qc.draw(output='text').single_string()
        except Exception:
            ascii_drawing = str(qc.draw(output='text'))
        with open(txt_path, 'w') as f:
            f.write(ascii_drawing)
        print(f"📝 Saved ASCII fallback to {os.path.abspath(txt_path)}")
        return txt_path

def run_double_slit(shots=1024, save=True):
    """
    1927 — Double-slit (single qubit superposition).
    Returns: (fig, histogram_path, circuit_path, {'counts': counts})
    """
    if _BACKEND is None:
        raise RuntimeError("No Aer backend available. Install qiskit Aer.")

    qc = QuantumCircuit(1, 1)
    qc.h(0)
    qc.measure(0, 0)

    # execute
    compiled = transpile(qc, _BACKEND)
    job = _BACKEND.run(compiled, shots=shots)
    result = job.result()
    counts = result.get_counts()

    # Save histogram + circuit drawing
    hist_path = os.path.join(ASSETS_DIR, 'double_slit_hist.png') if save else None
    circuit_path = os.path.join(ASSETS_DIR, 'double_slit_circuit.png') if save else None

    fig = None
    if save and hist_path:
        fig = _save_histogram(counts, hist_path)
    else:
        fig = plot_histogram(counts)

    if save and circuit_path:
        _save_circuit(qc, circuit_path)

    return fig, hist_path, circuit_path, {'counts': counts}


def run_cat(shots=1, dist_shots=1024, save=True):
    """
    1935 — Schrödinger's Cat (use one qubit measurement as the 'box').
    shots: number of shots for the actual 'single measurement' (default 1).
    dist_shots: number of shots used to create a distribution/histogram for visualization.
    Returns: (fig_dist, hist_path, circuit_path, result_str)
    """
    if _BACKEND is None:
        raise RuntimeError("No Aer backend available. Install qiskit Aer.")

    # Circuit for the single-shot experiment
    qc = QuantumCircuit(1, 1)
    qc.h(0)          # create superposition
    qc.measure(0, 0)

    # Single-shot measurement to decide alive/dead
    compiled_single = transpile(qc, _BACKEND)
    job_single = _BACKEND.run(compiled_single, shots=shots)
    res_single = job_single.result()
    counts_single = res_single.get_counts()
    # get single-shot outcome (if shots>1 returns counts; we interpret by sampling)
    single_key = next(iter(counts_single.keys()))
    result = "Alive 🟢" if single_key == '0' else "Dead 🔴"

    # Distribution for visualization
    compiled_dist = transpile(qc, _BACKEND)
    job_dist = _BACKEND.run(compiled_dist, shots=dist_shots)
    res_dist = job_dist.result()
    counts_dist = res_dist.get_counts()

    hist_path = os.path.join(ASSETS_DIR, 'cat_distribution_hist.png') if save else None
    circuit_path = os.path.join(ASSETS_DIR, 'cat_circuit.png') if save else None

    fig = None
    if save and hist_path:
        fig = _save_histogram(counts_dist, hist_path)
    else:
        fig = plot_histogram(counts_dist)

    if save and circuit_path:
        _save_circuit(qc, circuit_path)

    return fig, hist_path, circuit_path, result


def run_bell(shots=1024, save=True):
    """
    1964 — Bell pair (entanglement).
    Produces a Bell state and measures both qubits. Expect correlated 00/11.
    Returns: (fig, hist_path, circuit_path, {'counts': counts, 'correlation': percentage})
    """
    if _BACKEND is None:
        raise RuntimeError("No Aer backend available. Install qiskit Aer.")

    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])

    compiled = transpile(qc, _BACKEND)
    job = _BACKEND.run(compiled, shots=shots)
    res = job.result()
    counts = res.get_counts()

    # compute simple correlation metric (sum of 00 and 11)
    total = sum(counts.values())
    corr = 0
    for k, v in counts.items():
        if k in ['00', '11']:
            corr += v
    correlation_percent = 100.0 * corr / total if total > 0 else 0.0

    hist_path = os.path.join(ASSETS_DIR, 'bell_hist.png') if save else None
    circuit_path = os.path.join(ASSETS_DIR, 'bell_circuit.png') if save else None

    fig = None
    if save and hist_path:
        fig = _save_histogram(counts, hist_path)
    else:
        fig = plot_histogram(counts)

    if save and circuit_path:
        _save_circuit(qc, circuit_path)

    return fig, hist_path, circuit_path, {'counts': counts, 'correlation_percent': correlation_percent}


def run_future(n_qubits=3, save=True, headlines=None):
    """
    2125 — Quantum Random Future
    Use n_qubits to produce a random integer (0 .. 2^n_qubits - 1) and pick a headline.
    Returns: (fig, hist_path, circuit_path, {'random_int': val, 'headline': headline})
    """
    if headlines is None:
        headlines = [
            "🧠 Quantum AI proposes a new unifying theory of physics.",
            "🌍 Global Quantum Internet connects every continent.",
            "🔐 Quantum cryptography secures all global payments.",
            "🚀 Quantum-enhanced propulsion enables crewed interstellar probes.",
            "🧬 Quantum sensors map the human brain at synapse resolution.",
            "🏙️ Cities run on fault-tolerant quantum optimization."
        ]

    if _BACKEND is None:
        raise RuntimeError("No Aer backend available. Install qiskit Aer.")

    qc = QuantumCircuit(n_qubits, n_qubits)
    for q in range(n_qubits):
        qc.h(q)     # put each qubit in superposition
    qc.measure(range(n_qubits), range(n_qubits))

    compiled = transpile(qc, _BACKEND)
    job = _BACKEND.run(compiled, shots=1)
    res = job.result()
    counts = res.get_counts()
    # counts will have one key like '101' (order depends on qiskit version)
    outcome = next(iter(counts.keys()))
    # Convert binary string to int (qiskit orders may be little-endian -> try both)
    try:
        val = int(outcome, 2)
    except ValueError:
        # fallback: reverse string
        val = int(outcome[::-1], 2)

    headline = headlines[val % len(headlines)]

    # For visualization, create a distribution histogram (many shots)
    compiled_dist = transpile(qc, _BACKEND)
    dist_job = _BACKEND.run(compiled_dist, shots=1024)
    dist_res = dist_job.result()
    dist_counts = dist_res.get_counts()
    hist_path = os.path.join(ASSETS_DIR, 'future_distribution_hist.png') if save else None
    circuit_path = os.path.join(ASSETS_DIR, 'future_circuit.png') if save else None

    fig = None
    if save and hist_path:
        fig = _save_histogram(dist_counts, hist_path)
    else:
        fig = plot_histogram(dist_counts)

    if save and circuit_path:
        _save_circuit(qc, circuit_path)

    return fig, hist_path, circuit_path, {'random_int': val, 'headline': headline}


# Small __main__ test block (runs if executed directly)
if __name__ == "__main__":
    print("Quick tests (this runs when module executed directly)...")
    f, h, c, r = run_double_slit()
    print("Double slit counts:", r['counts'])
    f, h, c, r = run_cat()
    print("Cat result:", r)
    f, h, c, r = run_bell()
    print("Bell correlation %:", r['correlation_percent'])
    f, h, c, r = run_future()
    print("Future:", r)


Overwriting /content/quantum_time_capsule/quantum_scenes.py


In [ ]:
!pip install --upgrade pylatexenc qiskit


In [ ]:
%%writefile /content/quantum_time_capsule/quantum_scenes.py

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib
matplotlib.use("Agg")  # non-GUI backend
import matplotlib.pyplot as plt
import os


# -----------------------------
# Backend setup

try:
    _BACKEND = Aer.get_backend('aer_simulator')
except Exception:
    _BACKEND = Aer.get_backend('qasm_simulator')

BASE_DIR = os.environ.get('QT_CAPSULE_DIR', '/content/quantum_time_capsule')
ASSETS_DIR = os.path.join(BASE_DIR, 'assets')
os.makedirs(ASSETS_DIR, exist_ok=True)

# -----------------------------
# Utilities
# -----------------------------
def _save_histogram(counts, filepath):
    try:
        fig = plot_histogram(counts)
        fig.savefig(filepath, bbox_inches='tight')
        plt.close(fig)
        return fig
    except Exception:
        labels = list(counts.keys())
        values = [counts[k] for k in labels]
        fig, ax = plt.subplots()
        ax.bar(labels, values)
        ax.set_xlabel('Outcome')
        ax.set_ylabel('Counts')
        plt.tight_layout()
        fig.savefig(filepath)
        plt.close(fig)
        return fig

def _save_circuit(qc, filepath):
    """
    Save a quantum circuit as a PNG using Qiskit's mpl drawer.
    Requires pylatexenc.
    """
    fig = qc.draw(output='mpl', scale=1.2)
    fig.savefig(filepath, bbox_inches='tight')
    fig.clf()
    return filepath



# -----------------------------
# Experiments
# -----------------------------
def run_double_slit(shots=1024, save=True):
    qc = QuantumCircuit(1, 1)
    qc.h(0)
    qc.measure(0, 0)

    compiled = transpile(qc, _BACKEND)
    job = _BACKEND.run(compiled, shots=shots)
    counts = job.result().get_counts()

    hist_path = os.path.join(ASSETS_DIR, 'double_slit_hist.png') if save else None
    circuit_path = os.path.join(ASSETS_DIR, 'double_slit_circuit.png') if save else None

    fig = _save_histogram(counts, hist_path) if save and hist_path else plot_histogram(counts)
    if save and circuit_path:
        circuit_path = _save_circuit(qc, circuit_path)

    return fig, hist_path, circuit_path, {'counts': counts}

def run_cat(shots=1, dist_shots=1024, save=True):
    qc = QuantumCircuit(1,1)
    qc.h(0)
    qc.measure(0,0)

    # Single-shot for alive/dead
    compiled_single = transpile(qc, _BACKEND)
    job_single = _BACKEND.run(compiled_single, shots=shots)
    counts_single = job_single.result().get_counts()
    single_key = next(iter(counts_single.keys()))
    result = "Alive 🟢" if single_key == '0' else "Dead 🔴"

    # Distribution for histogram
    compiled_dist = transpile(qc, _BACKEND)
    counts_dist = _BACKEND.run(compiled_dist, shots=dist_shots).result().get_counts()

    hist_path = os.path.join(ASSETS_DIR, 'cat_distribution_hist.png') if save else None
    circuit_path = os.path.join(ASSETS_DIR, 'cat_circuit.png') if save else None

    fig = _save_histogram(counts_dist, hist_path) if save and hist_path else plot_histogram(counts_dist)
    if save and circuit_path:
        circuit_path = _save_circuit(qc, circuit_path)

    return fig, hist_path, circuit_path, result

def run_bell(shots=1024, save=True):
    qc = QuantumCircuit(2,2)
    qc.h(0)
    qc.cx(0,1)
    qc.measure([0,1],[0,1])

    compiled = transpile(qc, _BACKEND)
    counts = _BACKEND.run(compiled, shots=shots).result().get_counts()

    total = sum(counts.values())
    corr = sum(v for k,v in counts.items() if k in ['00','11'])
    correlation_percent = 100.0 * corr / total if total > 0 else 0.0

    hist_path = os.path.join(ASSETS_DIR, 'bell_hist.png') if save else None
    circuit_path = os.path.join(ASSETS_DIR, 'bell_circuit.png') if save else None

    fig = _save_histogram(counts, hist_path) if save and hist_path else plot_histogram(counts)
    if save and circuit_path:
        circuit_path = _save_circuit(qc, circuit_path)

    return fig, hist_path, circuit_path, {'counts': counts, 'correlation_percent': correlation_percent}

def run_future(n_qubits=3, save=True, headlines=None):
    if headlines is None:
        headlines = [
            "🧠 Quantum AI proposes a new unifying theory of physics.",
            "🌍 Global Quantum Internet connects every continent.",
            "🔐 Quantum cryptography secures all global payments.",
            "🚀 Quantum-enhanced propulsion enables crewed interstellar probes.",
            "🧬 Quantum sensors map the human brain at synapse resolution.",
            "🏙️ Cities run on fault-tolerant quantum optimization."
        ]

    qc = QuantumCircuit(n_qubits, n_qubits)
    for q in range(n_qubits):
        qc.h(q)
    qc.measure(range(n_qubits), range(n_qubits))

    compiled = transpile(qc, _BACKEND)
    outcome = next(iter(_BACKEND.run(compiled, shots=1).result().get_counts().keys()))

    try:
        val = int(outcome, 2)
    except ValueError:
        val = int(outcome[::-1], 2)

    headline = headlines[val % len(headlines)]

    compiled_dist = transpile(qc, _BACKEND)
    dist_counts = _BACKEND.run(compiled_dist, shots=1024).result().get_counts()

    hist_path = os.path.join(ASSETS_DIR, 'future_distribution_hist.png') if save else None
    circuit_path = os.path.join(ASSETS_DIR, 'future_circuit.png') if save else None

    fig = _save_histogram(dist_counts, hist_path) if save and hist_path else plot_histogram(dist_counts)
    if save and circuit_path:
        circuit_path = _save_circuit(qc, circuit_path)

    return fig, hist_path, circuit_path, {'random_int': val, 'headline': headline}

# -----------------------------
# Quick test block
# -----------------------------
if __name__ == "__main__":
    f, h, c, r = run_double_slit()
    print("Double slit counts:", r['counts'])
    f, h, c, r = run_cat()
    print("Cat result:", r)
    f, h, c, r = run_bell()
    print("Bell correlation %:", r['correlation_percent'])
    f, h, c, r = run_future()
    print("Future:", r)



Overwriting /content/quantum_time_capsule/quantum_scenes.py


In [ ]:
!pip install qiskit qiskit-aer --upgrade


In [ ]:
# ==============================================================
# quantum_scenes.py
# Person 1 — Quantum Logic Engineer
# --------------------------------------------------------------
# This module contains all the quantum circuit functions for
# the "Quantum Time Capsule: 100 Years of Quantum" hackathon project.
#
# Scenes implemented:
#  - 1927: Double Slit (Hadamard superposition)
#  - 1935: Schrödinger’s Cat (Random outcome)
#  - 1964: Bell State (Entanglement)
#  - 2125: Quantum Future (Quantum random headline generator)
#
# Each function returns either counts, outcomes, or generated content,
# and saves any plots to the /assets folder inside the project.
# ==============================================================

# --- Standard Imports ---
import os
import random
import matplotlib.pyplot as plt

# --- Qiskit Imports ---
# QuantumCircuit: to build circuits
# transpile: to prepare circuits for simulators
from qiskit import QuantumCircuit, transpile

# AerSimulator is the correct modern import for Qiskit Aer
from qiskit_aer import AerSimulator

# plot_histogram: to visualize measurement results
from qiskit.visualization import plot_histogram


# ==============================================================
# Ensure assets folder exists for saving images
# ==============================================================

ASSETS_DIR = '/content/quantum_time_capsule/assets'
os.makedirs(ASSETS_DIR, exist_ok=True)


# ==============================================================
# 1927: Double Slit Simulation
# ==============================================================

def run_double_slit():
    """
    Simulate a single-qubit Hadamard gate followed by measurement.
    This represents the famous double-slit experiment, showing
    a superposition of |0> and |1>.

    Returns:
        counts (dict): Measurement results from simulator.
        fig_path (str): Path to saved histogram image.
    """

    # Step 1: Create a circuit with 1 qubit and 1 classical bit
    qc = QuantumCircuit(1, 1)

    # Step 2: Apply Hadamard to create superposition
    qc.h(0)

    # Step 3: Measure the qubit
    qc.measure_all()

    # Step 4: Run on AerSimulator
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    result = simulator.run(tqc).result()

    # Step 5: Get measurement counts
    counts = result.get_counts()

    # Step 6: Plot histogram and save it to assets folder
    fig = plot_histogram(counts)
    fig_path = os.path.join(ASSETS_DIR, 'double_slit.png')
    plt.savefig(fig_path)
    plt.close()

    return counts, fig_path


# ==============================================================
# 1935: Schrödinger’s Cat Simulation
# ==============================================================

def run_cat():
    """
    Represent Schrödinger's cat thought experiment.
    Instead of a real quantum circuit, we simulate the randomness
    of measurement: Alive or Dead outcome.

    Returns:
        outcome (str): Either "Alive 🐱" or "Dead 💀"
    """
    outcome = random.choice(['Alive 🐱', 'Dead 💀'])
    return outcome


# ==============================================================
# 1964: Bell State (Entanglement) Simulation
# ==============================================================

def run_bell():
    """
    Create and measure a Bell state between two qubits.
    This represents quantum entanglement and Bell's inequality experiments.

    Returns:
        counts (dict): Measurement results from simulator.
        fig_path (str): Path to saved histogram image.
    """

    # Step 1: Create a 2-qubit circuit with 2 classical bits
    qc = QuantumCircuit(2, 2)

    # Step 2: Apply Hadamard on qubit 0
    qc.h(0)

    # Step 3: Apply CNOT (CX) between qubit 0 (control) and 1 (target)
    qc.cx(0, 1)

    # Step 4: Measure both qubits
    qc.measure_all()

    # Step 5: Run on AerSimulator
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    result = simulator.run(tqc).result()

    # Step 6: Get measurement counts
    counts = result.get_counts()

    # Step 7: Plot histogram and save it to assets folder
    fig = plot_histogram(counts)
    fig_path = os.path.join(ASSETS_DIR, 'bell_state.png')
    plt.savefig(fig_path)
    plt.close()

    return counts, fig_path


# ==============================================================
# 2125: Quantum Future Headline Generator
# ==============================================================

def run_future():
    """
    Generate a futuristic 'quantum news headline' using randomness.
    This simulates a quantum random number generator picking a future outcome.

    Returns:
        headline (str): A randomly chosen future quantum breakthrough.
    """

    headlines = [
        "🌐 Quantum Internet spans the globe!",
        "🧠 Quantum AI surpasses classical limits.",
        "🚀 Humans teleport information to Mars.",
        "🔐 Quantum encryption secures global finance.",
        "🤖 Hybrid Quantum-Classical superintelligence emerges.",
        "⚛️ Quantum sensors map the universe in real-time.",
        "🏥 Quantum biology unlocks disease cures."
    ]

    headline = random.choice(headlines)
    return headline


# ==============================================================
# Quick Self-Test (Optional)
# --------------------------------------------------------------
# You can run this file directly to test each function individually.
# ==============================================================

if __name__ == "__main__":
    print("Running self-test for quantum_scenes.py ...\n")

    # Test 1927 scene
    counts_1927, path_1927 = run_double_slit()
    print("1927 Double Slit Counts:", counts_1927)
    print("Saved Histogram:", path_1927, "\n")

    # Test 1935 scene
    cat_outcome = run_cat()
    print("1935 Schrödinger’s Cat Outcome:", cat_outcome, "\n")

    # Test 1964 scene
    counts_bell, path_bell = run_bell()
    print("1964 Bell State Counts:", counts_bell)
    print("Saved Histogram:", path_bell, "\n")

    # Test 2125 scene
    future_headline = run_future()
    print("2125 Quantum Future Headline:", future_headline, "\n")

    print("✅ All functions executed successfully.")


Running self-test for quantum_scenes.py ...

1927 Double Slit Counts: {'1 0': 517, '0 0': 507}
Saved Histogram: /content/quantum_time_capsule/assets/double_slit.png 

1935 Schrödinger’s Cat Outcome: Alive 🐱 

1964 Bell State Counts: {'00 00': 537, '11 00': 487}
Saved Histogram: /content/quantum_time_capsule/assets/bell_state.png 

2125 Quantum Future Headline: 🚀 Humans teleport information to Mars. 

✅ All functions executed successfully.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#==================================================================================================================================================================================================================================================================================================================================================================
#================================================================================================================================================

In [ ]:
!pip install pyngrok streamlit > /dev/null

In [ ]:
# Create the folder structure if it doesn't exist
!mkdir -p /content/quantum_time_capsule


In [ ]:
!mkdir -p /content/quantum_time_capsule/assets


In [ ]:
!pip install qiskit qiskit-aer --quiet


In [ ]:
!pip install streamlit pyngrok qiskit qiskit-aer matplotlib


In [ ]:
import os

paths = [
    "/content/quantum_time_capsule",
    "/content/quantum_time_capsule/app.py",
    "/content/quantum_time_capsule/quantum_scenes.py",
    "/content/quantum_time_capsule/assets"
]

for p in paths:
    print(f"{p}: {'✅ Exists' if os.path.exists(p) else '❌ Missing'}")



/content/quantum_time_capsule: ✅ Exists
/content/quantum_time_capsule/app.py: ✅ Exists
/content/quantum_time_capsule/quantum_scenes.py: ✅ Exists
/content/quantum_time_capsule/assets: ✅ Exists


In [ ]:
import os
os.makedirs('/content/quantum_time_capsule/assets', exist_ok=True)
print("✅ assets directory exists:", os.path.exists('/content/quantum_time_capsule/assets'))


✅ assets directory exists: True


In [ ]:
import sys, os

# Add your project folder to Python's path
PROJECT_DIR = "/content/quantum_time_capsule"
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)

# Now try importing the module
from quantum_scenes import run_double_slit

# Test that everything works
fig, hist, circ, res = run_double_slit()
print("Paths:", hist, circ)
print("Exist:", os.path.exists(hist), os.path.exists(circ))


Paths: /content/quantum_time_capsule/assets/double_slit_hist.png /content/quantum_time_capsule/assets/double_slit_circuit.png
Exist: True True


In [ ]:
%pip install qiskit pylatexenc matplotlib


In [ ]:
%%writefile /content/quantum_time_capsule/app.py
import os
import sys
import time
import streamlit as st
import importlib
import matplotlib
matplotlib.use("Agg")
from datetime import datetime

# ==============================================================
# Setup
# ==============================================================
PROJECT_DIR = '/content/quantum_time_capsule'
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)

try:
    import quantum_scenes
    importlib.reload(quantum_scenes)
    from quantum_scenes import run_double_slit, run_cat, run_bell, run_future
except Exception as e:
    st.set_page_config(page_title="Quantum Time Capsule", page_icon="⚛️")
    st.error(f"Error importing quantum_scenes: {e}")
    st.stop()

st.set_page_config(page_title="Quantum Time Capsule", page_icon="⚛️", layout="wide")

# ==============================================================
# Styles
# ==============================================================
st.markdown("""
<style>
.stApp {
  background: linear-gradient(135deg, #ff66cc, #7b2ff7);
  color: white;
  font-family: 'Poppins', sans-serif;
}
.stButton>button {
  background: #fbb6ce;
  color: #000;
  border: none;
  border-radius: 999px;
  font-weight: bold;
  padding: 0.6rem 1.2rem;
  transition: all 0.25s ease-in-out;
}
.stButton>button:hover {
  background: #fff;
  color: #ff66cc;
  box-shadow: 0 0 25px rgba(255,255,255,0.4);
}
</style>
""", unsafe_allow_html=True)

# ==============================================================
# State control
# ==============================================================
pages = ["intro", "double_slit", "cat", "bell", "future", "outro"]
if "page" not in st.session_state:
    st.session_state.page = "intro"

def go_to(page):
    st.session_state.page = page

def wait_for_file(path, timeout=2):
    """Wait for a file to appear (avoid race conditions)."""
    start = time.time()
    while time.time() - start < timeout:
        if os.path.exists(path):
            return True
        time.sleep(0.1)
    return False

def show_narration_bar(audio_path):
    """Show a persistent audio bar for narration."""
    if os.path.exists(audio_path):
        st.audio(audio_path, format="audio/mp3")
    else:
        st.warning(f"🎧 Audio file not found: {audio_path}")


def narration_controls(audio_path, label="🔊 Play Narration"):
    """Show a Play Narration button and play if clicked or autoplay fails."""
    if os.path.exists(audio_path):
        # Try autoplay (hidden)
        st.markdown(
            f"""
            <audio id="narration" autoplay hidden>
                <source src="file://{audio_path}" type="audio/mp3">
            </audio>
            """,
            unsafe_allow_html=True
        )
        # Manual play fallback
        if st.button(label):
            st.audio(audio_path, format="audio/mp3")
    else:
        st.warning(f"🎧 Audio file not found: {audio_path}")


# ==============================================================
# PAGE LOGIC
# ==============================================================

page = st.session_state.page

# ---------- INTRO ----------
if page == "intro":
    st.title("🌌 Quantum Time Capsule")
    st.subheader("A Journey Through 100 Years of Quantum Discovery")
    st.write("Step into the Quantum Time Capsule — an immersive quantum storytelling experience powered by real quantum circuits.")
    st.image("https://upload.wikimedia.org/wikipedia/commons/6/6b/Bloch_sphere.svg", caption="Qubit on the Bloch Sphere")
    show_narration_bar("/content/quantum_time_capsule/assets/intro.mp3")

    if st.button("🚀 Begin Journey"):
        go_to("double_slit")

# ---------- DOUBLE SLIT ----------

elif page == "double_slit":
    st.header("1927 — The Double Slit Experiment")
    st.write("In 1927, the double slit experiment revealed that light behaves as both a wave and a particle")

    show_narration_bar("/content/quantum_time_capsule/assets/1927_double_slit.mp3")



    if st.button("Run Double Slit Simulation 🧪"):
        fig, hist_path, circuit_path, result = run_double_slit()
        # Histogram
        if hist_path and os.path.exists(hist_path):
            st.image(hist_path, caption="Quantum Interference Pattern", use_container_width=True)
        else:
            st.warning("Histogram file not found!")

        # Circuit
        if circuit_path and os.path.exists(circuit_path):
            st.image(circuit_path, caption="Quantum Circuit Diagram", use_container_width=True)
        else:
            st.warning("Circuit file not found!")

    col1, col2 = st.columns(2)
    with col1:
        if st.button("⬅ Back"):
            go_to("intro")
    with col2:
        if st.button("Next ➡ Schrödinger’s Cat"):
            go_to("cat")

# ---------- SCHRÖDINGER'S CAT ----------
elif page == "cat":
    st.header("1935 — Schrödinger's Cat 🐱")
    st.write("A cat in a box — simultaneously alive and dead until observed.")


    show_narration_bar("/content/quantum_time_capsule/assets/1935_schrodinger.mp3")


    if st.button("Open the Box 🎁"):
        fig, hist_path, circuit_path, result = run_cat()
        # Histogram
        if hist_path and os.path.exists(hist_path):
            st.image(hist_path, caption="Measurement Distribution", use_container_width=True)
        # Circuit
        if circuit_path and os.path.exists(circuit_path):
            st.image(circuit_path, caption="Circuit Diagram", use_container_width=True)
        st.success(f"The cat is: {result}")

    col1, col2 = st.columns(2)
    with col1:
        if st.button("⬅ Back"):
            go_to("double_slit")
    with col2:
        if st.button("Next ➡ Bell’s Theorem"):
            go_to("bell")

# ---------- BELL’S THEOREM ----------
elif page == "bell":
    st.header("1964 — Bell’s Theorem & Quantum Entanglement 🔗")
    st.write("Two particles become entangled — measuring one instantly affects the other.")

    show_narration_bar("/content/quantum_time_capsule/assets/1964_bell.mp3")



    if st.button("Run Bell Pair Simulation 🧬"):
        fig, hist_path, circuit_path, result = run_bell()
        if hist_path and os.path.exists(hist_path):
            st.image(hist_path, caption="Entanglement Measurement Results", use_container_width=True)
        if circuit_path and os.path.exists(circuit_path):
            st.image(circuit_path, caption="Circuit Diagram", use_container_width=True)
        st.json(result["counts"])
        st.metric("Correlation", f"{result['correlation_percent']:.2f}%")

    col1, col2 = st.columns(2)
    with col1:
        if st.button("⬅ Back"):
            go_to("cat")
    with col2:
        if st.button("Next ➡ Quantum Future"):
            go_to("future")


# ---------- FUTURE ----------
elif page == "future":
    st.header("2125 — The Quantum Future 🚀")
    st.write("Quantum technologies dominate our world. Let's see what fate awaits...")


    show_narration_bar("/content/quantum_time_capsule/assets/2125_future.mp3")


    if st.button("Generate Future Headline 📰"):
        fig, hist_path, circuit_path, result = run_future()
        if hist_path and os.path.exists(hist_path):
            st.image(hist_path, caption="Quantum Random Generator Distribution", use_container_width=True)
        if circuit_path and os.path.exists(circuit_path):
            st.image(circuit_path, caption="Quantum Circuit Diagram", use_container_width=True)
        st.info(result["headline"])

    col1, col2 = st.columns(2)
    with col1:
        if st.button("⬅ Back"):
            go_to("bell")
    with col2:
        if st.button("Next ➡ Outro"):
            go_to("outro")


# ---------- OUTRO ----------
elif page == "outro":
    st.header("✨ End of Journey")
    st.write("""
    You've completed a century-long odyssey through the evolution of quantum theory —
    from the mysteries of interference to the promises of the quantum future.
    """)

    show_narration_bar("/content/quantum_time_capsule/assets/outro.mp3")


    if st.button("🔁 Restart Journey"):
        go_to("intro")

st.markdown("---")
st.caption(f"© {datetime.now().year} Quantum Time Capsule — Open Quantum Project ⚛️")


Overwriting /content/quantum_time_capsule/app.py


In [ ]:
import os
print("Files in assets folder:")
print(os.listdir("/content/quantum_time_capsule/assets"))


Files in assets folder:
['double_slit_hist.png', 'bell_state.png', 'double_slit_circuit.png', 'double_slit.png']


In [ ]:
!pip install pylatexenc matplotlib qiskit qiskit-aer --quiet


In [ ]:
!pip install streamlit pyngrok qiskit matplotlib


In [ ]:
!pip install qiskit qiskit-aer --quiet


In [ ]:
#==================================================================================================================================================================================================================================================================================================================================================================
#================================================================================================================================================

In [ ]:
!pip install pyngrok --quiet
!pip install streamlit pyngrok --quiet


In [ ]:
!pip install streamlit --quiet


In [ ]:
!pip install --upgrade qiskit pylatexenc


In [ ]:
import pylatexenc
print("pylatexenc version:", pylatexenc.__version__)


pylatexenc version: 2.10


In [ ]:
# --- Reset ngrok configuration and sessions ---
!pkill ngrok || echo "No ngrok process running."

!rm -rf /root/.ngrok2 /root/.config/ngrok
!pip install pyngrok --quiet

from pyngrok import ngrok
import time

# --- Re-add your token ---
!ngrok config add-authtoken "349JeEDuwzrP28lSdnEwT3gpWbM_5QEXD4gzVzC6bsphxe2oF"

print("✅ ngrok reset and re-authenticated.")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ ngrok reset and re-authenticated.


In [ ]:
# Run Streamlit quietly in the background
get_ipython().system_raw("streamlit run /content/quantum_time_capsule/app.py --server.port 8507 &")

time.sleep(5)
print("✅ Streamlit app is running in the background.")


✅ Streamlit app is running in the background.


In [ ]:
from pyngrok import ngrok

# Connect ngrok to Streamlit port
public_url = ngrok.connect(8507)

# Just show the clean clickable link
print(f"\n🌍 CLICK HERE to open your Quantum Time Capsule app:\n{public_url.public_url}")



🌍 CLICK HERE to open your Quantum Time Capsule app:
https://imelda-hyperemic-tumidly.ngrok-free.dev


In [ ]:
!mkdir -p /content/quantum_time_capsule/assets


In [ ]:
%%writefile /content/quantum_time_capsule/narration.py
from gtts import gTTS
import os



# --- Narration texts for each quantum scene ---
narrations = {
    "intro": "Welcome to the Quantum Time Capsule. Journey with us through a hundred years of quantum discovery.",
    "1927_double_slit": "In 1927, the double slit experiment revealed that light behaves as both a wave and a particle.",
    "1935_schrodinger": "In 1935, Erwin Schrödinger imagined a cat that was both dead and alive, illustrating quantum superposition.",
    "1964_bell": "In 1964, John Bell proposed his famous theorem, showing that quantum mechanics defies classical ideas of locality.",
    "2125_future": "In 2125, quantum computers shape our world, solving problems once thought impossible.",
    "outro": "Congratulations, traveler. You have witnessed the milestones of quantum history. The journey continues into the unknown future."
}


# --- Convert each narration to audio and save in the project assets folder ---
ASSETS_DIR = "/content/quantum_time_capsule/assets"
os.makedirs(ASSETS_DIR, exist_ok=True)

for key, text in narrations.items():
    tts = gTTS(text)
    filename = os.path.join(ASSETS_DIR, f"{key}.mp3")
    tts.save(filename)
    print(f"✅ Saved: {filename}")


Overwriting /content/quantum_time_capsule/narration.py


In [ ]:
!python /content/quantum_time_capsule/narration.py


✅ Saved: /content/quantum_time_capsule/assets/intro.mp3
✅ Saved: /content/quantum_time_capsule/assets/1927_double_slit.mp3
✅ Saved: /content/quantum_time_capsule/assets/1935_schrodinger.mp3
✅ Saved: /content/quantum_time_capsule/assets/1964_bell.mp3
✅ Saved: /content/quantum_time_capsule/assets/2125_future.mp3
✅ Saved: /content/quantum_time_capsule/assets/outro.mp3


In [ ]:
import os
os.listdir("/content/quantum_time_capsule/assets")



['double_slit_hist.png',
 '1927_double_slit.mp3',
 'outro.mp3',
 'intro.mp3',
 '1935_schrodinger.mp3',
 'bell_state.png',
 '2125_future.mp3',
 'double_slit_circuit.png',
 'double_slit.png',
 '1964_bell.mp3']

In [ ]:
!mkdir -p quantum_time_capsule/assets
%cd quantum_time_capsule


/content/quantum_time_capsule


In [ ]:
!pip install qiskit qiskit-aer matplotlib --quiet


In [ ]:
%cd /content/quantum_time_capsule


/content/quantum_time_capsule


In [ ]:
!pip install qiskit-aer


In [ ]:
!pip install --upgrade qiskit qiskit-aer matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 65.1 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.0
    Uninstalling matplotlib-3.10.0:
      Successfully uninstalled matplotlib-3.10.0


In [ ]:
!pip uninstall -y qiskit qiskit-aer


Found existing installation: qiskit 2.2.2
Uninstalling qiskit-2.2.2:
  Successfully uninstalled qiskit-2.2.2
Found existing installation: qiskit-aer 0.17.2
Uninstalling qiskit-aer-0.17.2:
  Successfully uninstalled qiskit-aer-0.17.2


In [ ]:
!pip install qiskit qiskit-aer matplotlib --upgrade


ERROR: Operation cancelled by user
^C


In [ ]:
!pip install qiskit matplotlib --quiet


In [ ]:
%%writefile /content/quantum_time_capsule/quantum_scenes.py

from qiskit import QuantumCircuit, transpile, execute
from qiskit import BasicAer  # <- fallback simulator included in core qiskit
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import os
import random

# Assets folder
ASSETS_DIR = "/content/quantum_time_capsule/assets"
os.makedirs(ASSETS_DIR, exist_ok=True)

# Backend: use BasicAer qasm_simulator
BACKEND = BasicAer.get_backend('qasm_simulator')

# Helper: save histogram
def _save_histogram(counts, filename):
    fig = plot_histogram(counts)
    path = os.path.join(ASSETS_DIR, filename)
    fig.savefig(path, bbox_inches='tight')
    plt.close()
    return path

# 1927: Double Slit
def run_double_slit(shots=1024):
    qc = QuantumCircuit(1,1)
    qc.h(0)
    qc.measure_all()
    job = execute(qc, BACKEND, shots=shots)
    counts = job.result().get_counts()
    path = _save_histogram(counts, "double_slit.png")
    return None, path, None, {"counts": counts}

# 1935: Schrödinger's Cat
def run_cat(shots=1):
    qc = QuantumCircuit(1,1)
    qc.h(0)
    qc.measure_all()
    job = execute(qc, BACKEND, shots=shots)
    counts = job.result().get_counts()
    result = "Alive 🟢" if list(counts.keys())[0] == "0" else "Dead 🔴"
    path = _save_histogram(counts, "cat.png")
    return None, path, None, result

# 1964: Bell State
def run_bell(shots=1024):
    qc = QuantumCircuit(2,2)
    qc.h(0)
    qc.cx(0,1)
    qc.measure_all()
    job = execute(qc, BACKEND, shots=shots)
    counts = job.result().get_counts()
    path = _save_histogram(counts, "bell.png")
    correlation = 100*sum(counts[k] for k in counts if k in ["00","11"])/sum(counts.values())
    return None, path, None, {"counts": counts, "correlation_percent": correlation}

# 2125: Quantum Future
def run_future(n_qubits=3):
    headlines = [
        "🧠 Quantum AI creates new physics models.",
        "🌍 Quantum Internet connects continents.",
        "🔐 Quantum security is unbreakable.",
        "🚀 Quantum travel possible.",
        "🧬 Brain-mapping via quantum sensors.",
        "🏙️ Cities optimized by quantum computers."
    ]
    qc = QuantumCircuit(n_qubits,n_qubits)
    for q in range(n_qubits):
        qc.h(q)
    qc.measure_all()
    job = execute(qc, BACKEND, shots=1)
    outcome = list(job.result().get_counts().keys())[0]
    val = int(outcome,2)
    headline = headlines[val % len(headlines)]
    job2 = execute(qc, BACKEND, shots=1024)
    counts = job2.result().get_counts()
    path = _save_histogram(counts, "future.png")
    return None, path, None, {"random_int": val, "headline": headline}


In [ ]:
import os
os.getcwd()


In [ ]:
%cd /content/quantum_time_capsule


In [ ]:
!ls


In [ ]:
import sys
if "/content/quantum_time_capsule" not in sys.path:
    sys.path.append("/content/quantum_time_capsule")


In [ ]:
!pip install qiskit-aer --upgrade


In [ ]:
!pip uninstall -y qiskit qiskit-aer


In [ ]:
!pip install qiskit==0.42.1 qiskit-aer==0.12.1


In [ ]:
!python --version


In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.10 python3.10-distutils -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo update-alternatives --config python3
!sudo apt-get install python3.10-venv -y


In [ ]:
!pip install --upgrade pip
!pip install qiskit qiskit-aer matplotlib


In [ ]:
!pip install qiskit


In [ ]:
%%writefile /content/quantum_time_capsule/quantum_scenes.py
# /content/quantum_time_capsule/quantum_scenes.py

from qiskit import QuantumCircuit, transpile, execute, BasicAer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import os, random

# Use BasicAer simulator
_BACKEND = BasicAer.get_backend('qasm_simulator')

# Ensure assets folder exists
ASSETS_DIR = "/content/quantum_time_capsule/assets"
os.makedirs(ASSETS_DIR, exist_ok=True)

# Helper function to save histogram
def _save_histogram(counts, filepath):
    fig = plot_histogram(counts)
    fig.savefig(filepath, bbox_inches='tight')
    plt.close()
    return fig

# 1927 Double Slit
def run_double_slit(shots=1024):
    qc = QuantumCircuit(1,1)
    qc.h(0)
    qc.measure_all()
    job = execute(qc, backend=_BACKEND, shots=shots)
    counts = job.result().get_counts()
    path = os.path.join(ASSETS_DIR, "double_slit.png")
    _save_histogram(counts, path)
    return None, path, None, {"counts": counts}

# 1935 Schrödinger's Cat
def run_cat(shots=1):
    qc = QuantumCircuit(1,1)
    qc.h(0)
    qc.measure_all()
    job = execute(qc, backend=_BACKEND, shots=shots)
    counts = job.result().get_counts()
    result = "Alive 🟢" if list(counts.keys())[0] == "0" else "Dead 🔴"
    path = os.path.join(ASSETS_DIR, "cat.png")
    _save_histogram(counts, path)
    return None, path, None, result

# 1964 Bell State
def run_bell(shots=1024):
    qc = QuantumCircuit(2,2)
    qc.h(0)
    qc.cx(0,1)
    qc.measure_all()
    job = execute(qc, backend=_BACKEND, shots=shots)
    counts = job.result().get_counts()
    path = os.path.join(ASSETS_DIR, "bell.png")
    _save_histogram(counts, path)
    correlation = 100*sum(counts[k] for k in counts if k in ["00","11"])/sum(counts.values())
    return None, path, None, {"counts": counts, "correlation_percent": correlation}

# 2125 Quantum Future
def run_future(n_qubits=3):
    headlines = [
        "🧠 Quantum AI creates new physics models.",
        "🌍 Quantum Internet connects continents.",
        "🔐 Quantum security is unbreakable.",
        "🚀 Quantum travel possible.",
        "🧬 Brain-mapping via quantum sensors.",
        "🏙️ Cities optimized by quantum computers."
    ]
    qc = QuantumCircuit(n_qubits,n_qubits)
    for q in range(n_qubits):
        qc.h(q)
    qc.measure_all()

    # Run once to pick a “future headline”
    job = execute(qc, backend=_BACKEND, shots=1)
    outcome = list(job.result().get_counts().keys())[0]
    val = int(outcome,2)
    headline = headlines[val % len(headlines)]

    # Run for histogram
    job2 = execute(qc, backend=_BACKEND, shots=1024)
    counts = job2.result().get_counts()
    path = os.path.join(ASSETS_DIR, "future.png")
    _save_histogram(counts, path)

    return None, path, None, headline


In [ ]:
import sys
PROJECT_DIR = "/content/quantum_time_capsule"
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)


In [ ]:
!pip install qiskit --upgrade
!pip install qiskit-aer --upgrade


In [ ]:
!pip install qiskit qiskit-aer matplotlib


In [ ]:
import os
from pyngrok import ngrok
import streamlit as st
from PIL import Image


In [ ]:
!pip install --upgrade qiskit qiskit-aer streamlit matplotlib pyngrok gTTS


In [ ]:
import os
from pyngrok import ngrok
import streamlit as st
from PIL import Image


In [ ]:
os.makedirs("assets", exist_ok=True)
